In [1]:
#import csv
import re
import codecs
#import unidecode
#import string
from nltk import FreqDist

File = "output_got.csv"
clean_data_file = "clean_data.txt"
clean_data = []; all_hashtags =[];

with codecs.open(clean_data_file, "w", "utf-8") as out_data:
    with open(File, "r", encoding = "utf8") as my_input_file:
        next(my_input_file)
        for line in my_input_file:
            # clean data
            line = re.sub("`|@|,|~|\n|'[ '|'] '|'?'|<.*>|<|!|\.|@|\$|\*|:|%|\+|…|\\\\|\/|«|»|···|\||\•|\?|\(|\)|=|-|&|;|\_|—|~|¯|\{|\}|\[|\]|£|€|¥|¿|–|\“|\”|\‘|\’|\"|,|'", "", line)
            line = re.sub("http[a-zA-Z0-9]+", "", line)
            line = line.lower()
            line = re.sub("pictwitter"," pictwitter", line);
            line = re.sub(" # "," #", line); line = re.sub("#"," #", line); line = re.sub("# ","", line); 
            for word in line.split(' '):
                if word.__contains__("pictwitter"): line.replace(word, '');
                if (word == '') or (len(word) < 2) : line.replace(word, '');
            line = re.sub("  "," ", line); line = line[:-2]
            clean_data.append(line)
            #line = unidecode.unidecode(line)
            out_data.write(line+'\n')

for line in clean_data:
    for word in line.split(' '):
        if len(word)> 1 :
            if word[0] == '#': all_hashtags.append(word);

hashtag_file = "hashtags.txt"
top_hashtags = FreqDist(all_hashtags).most_common(6)
tweets_file = "tweets.txt"
hashtags = [x[0] for x in top_hashtags]; hashtags = hashtags[1:]

with codecs.open(hashtag_file, "w", "utf-8") as hashfile:
    for tag in hashtags:
        hashfile.write(tag + '\n')


with codecs.open(tweets_file, "w", "utf-8") as tweets:
    for line in clean_data:
        if any(s in line for s in hashtags):
            tweets.write(line+'\n')

print('File Successfully written.')


File Successfully written.


In [ ]:
import codecs
#import string
import math
import re
from collections import Counter, defaultdict
#from spacy.lemmatizer import Lemmatizer as lemma
#from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
import pandas as pd
#import pickle
#import random
#import time
import numpy as np

htags_file = "hashtags.txt"
tweets_file = "tweets.txt"

hashtags = []; htags_tweets = {}
with codecs.open(htags_file, "r", "utf-8") as hashtagsFile:
    for word in hashtagsFile:
        tag = word.split('\n')[0]
        hashtags.append(tag);

# Dict = {Hashtag: Tweets}, stop words removal and bag of words
htags_tweets = dict.fromkeys(hashtags,'')
stop_words = set(stopwords.words('english'));
bow = set(); hashtweets = defaultdict(list); testHT = defaultdict(list);

with codecs.open(tweets_file, "r", "utf-8") as tweets:
    for tweet in tweets:
        #Stop words removal
        word_list = re.sub("`|@|,|~|\n|'[ '|'] '|'?'", " ", tweet).split(' ');
        filt_word_list = [word for word in word_list if (word not in stop_words) and (word != '' and len(word)>2) and (not word.__contains__('pictwittercom'))]
        filtered_tweet = ' '.join(filt_word_list)

        for tag in hashtags:
            if tag in filtered_tweet:
                flag = np.random.choice([0, 1], 1, p=[0.75, 0.25])
                if flag == 1:
                    #Test data
                    testHT[tag].append(filtered_tweet)
                else: 
                    # Train set: Dict = {Hashtag: Tweets}
                    hashtweets[tag].append(filtered_tweet) # {Hashtag: [T1, T2, T3]}
                    htags_tweets[tag] += re.sub('#' , '', filtered_tweet)

htpanda = pd.DataFrame.from_dict(hashtweets, orient='index')
testpanda = pd.DataFrame.from_dict(testHT, orient='index')

file_name = "trainHT.pkl"
htpanda.to_pickle(file_name)

test_file = "testHT.pkl"
testpanda.to_pickle(test_file)

for txt in htags_tweets.values():
    #bag of words: set creation
    bow = set(bow).union(set(txt.split(' ')))

#Bag of words initialization and updation for each class(hashtag)
bowDict = {}
for h in htags_tweets.keys():
    bowDict[h] = dict.fromkeys(bow, 0)

    for w in htags_tweets[h].split(' ') :
        bowDict[h][w] += 1

#Compute Term-Frequency
def computeTF(bowDict, tweet):
    tfDict = {}
    tweetlen = len(tweet.split(' '))
    for word, count in bowDict.items():
        tfDict[word] = count/float(tweetlen)
    return tfDict

for h, t in htags_tweets.items():
    tftweets = {}
    tftweets[h] = computeTF(bowDict[h], t)

#Compute Inverse document frequency
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    idfDict = htags_tweets.fromkeys(docList[0].keys(), 0)
    for doc in docList: #Each class(hashtag)
        for word, val in doc.items():
            if val > 0:
                if word not in idfDict.keys():
                    idfDict[word] = 1
                else :
                    idfDict[word] += 1

    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))

    return idfDict

idfs = computeIDF([bowDict[h] for h in bowDict.keys()])

#Compute TF-IDF
def computeTFIDF(tfhashtxt, idfs):
    tfidf = {}
    for word, val in tfhashtxt.items():
        tfidf[word] = val*idfs[word]
    return tfidf

tfidfBow = {}; topfivewords = {}
for hashtag in htags_tweets.keys():
    tfidfBow[hashtag] = computeTFIDF(bowDict[hashtag], idfs)
    topfivewords[hashtag] = dict(Counter(tfidfBow[hashtag]).most_common(40))

tfidf_file = "tfidf.txt"
tfidf = {}

with codecs.open(tfidf_file, "w", "utf-8") as out_data:
    for ht, rw in topfivewords.items():
        out_data.write(ht+ ' '+str(' '.join(rw.keys()) + '\n'))
        tfidf[ht] = rw.keys()

tfidfpd = pd.DataFrame.from_dict(tfidf, orient='index')

file_name = "tfidf.pkl"
tfidfpd.to_pickle(file_name)

print("Success!!!")


In [ ]:
#import numpy as np
import pandas as pd
import pickle
import time

"""To transform train and test dataset into input and output vectors 
for the multiclass-logistic regression."""

rank = pickle.load(open('tfidf.pkl', 'rb'));
trainXY = pickle.load(open('trainHT.pkl', 'rb'));\

bow = set()
for i in range(0, len(rank)):
    rankwords = rank.iloc[i]
    bow = set(bow).union(set(rankwords))

#Train data X Y
X = list(bow); Y = list(rank.index.get_values());
r, c = trainXY.shape;
train = {}; train[tuple([0]*len(X))] = 0;

start_time = time.time()
for i in range(r):
    outY = i

    for j in range(c):
        if trainXY.ix[i, j] != None :
            inX = [0]*len(X)

            for feature in X:
                if feature in trainXY.ix[i, j]:
                    inX[X.index(feature)] = 1
                    if tuple(inX) not in train:
                        train[tuple(inX)] = outY
        else : 
            break    

trainpd = pd.DataFrame.from_dict(train, orient='index')
trainpd.to_pickle("trainXY.pkl")  
elapsed_time = time.time() - start_time
print('Train data created',elapsed_time)

#Test data X Y
rank = pickle.load(open('tfidf.pkl', 'rb'));
bow = set()

for i in range(0, len(rank)):
    rankwords = rank.iloc[i]
    bow = set(bow).union(set(rankwords))
    
testXY = pickle.load(open('testHT.pkl', 'rb'));
X = list(bow); Y = list(rank.index.get_values());
r, c = testXY.shape;
test = {}; test[tuple([0]*len(X))] = 0;

start_time = time.time()
for i in range(r):
    outY = i

    for j in range(c):
        if testXY.ix[i, j] != None :
            inX = [0]*len(X)

            for feature in X:
                if feature in testXY.ix[i, j]:
                    inX[X.index(feature)] = 1
                    if tuple(inX) not in test:
                        test[tuple(inX)] = outY
        else : 
            break  
                
testpd = pd.DataFrame.from_dict(test, orient='index')
testpd.to_pickle("testXY.pkl") 
elapsed_time = time.time() - start_time
print('Test data created',elapsed_time)       


In [ ]:
import numpy as np
#import pandas as pd
import pickle
#import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_recall_fscore_support as score

#Train data
train = pickle.load(open('trainXY.pkl', 'rb'));# print(rank);
trainX = np.array(list(train.index.get_values()))

trainY = []
for j in range(len(train)):
    trainY.append(list(train.ix[j])[0])
trainY = np.array(trainY)

#Test data
test = pickle.load(open('testXY.pkl', 'rb'));# print(rank);
testX = np.array(list(test.index.get_values()))

testY = []
for j in range(len(test)):
    testY.append(list(test.ix[j])[0])
testY = np.array(testY)

#Logistic regression.
logreg = LogisticRegression( C=1e1, solver='lbfgs', multi_class='multinomial')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(trainX, trainY)
PredY = logreg.predict(testX)

print(confusion_matrix(testY, PredY))
accuracy = accuracy_score(testY, PredY)
print('Accuracy: ', accuracy)

"""
precision, recall, fscore, support = score(testY, PredY)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support)) """
